In [1]:
import polars as pl
from ipv6_scanner.config import *
from ipv6_scanner.filter import *
import datetime

2025-07-28 22:47:20.709 | INFO     | ipv6_scanner.config:<module>:11 - PROJ_ROOT path is: /home/mk/AG_Internetmessungen/TU-Dresden/Work/IPv6-Scanning-And-Sensing/artifacts-conext25-ipv6-scanner


In [ ]:
columns = ['Timestamp','scan_source_64','scan_source_128','Session_ID_128','Session_ID_64','dest_addr_type',
           'UDP_src_port','UDP_dst_port','TCP_src_port','TCP_dst_port','Protocol',
           'scantool','AS-Number','fullhex_destination_address','is_oneoff_128','period_128']

In [3]:
pl.enable_string_cache()

In [4]:
dft1 = pl.scan_parquet(T1_DATAFRAME).filter((pl.col('Timestamp')<pd.to_datetime('2024-07-03')))
dft2 = pl.scan_parquet(T2_DATAFRAME).filter((pl.col('Timestamp')<pd.to_datetime('2024-07-03')))
dft3 = pl.scan_parquet(T3_DATAFRAME).filter((pl.col('Timestamp')<pd.to_datetime('2024-07-03')))
dft4 = pl.scan_parquet(T4_DATAFRAME).filter((pl.col('Timestamp')<pd.to_datetime('2024-07-03')))
full_df = concat_frames([dft1,dft2,dft3,dft4],['T1','T2','T3','T4'],'Telescope',columns)

# Table 02

In [5]:
aggregated_df = full_df.select(['Timestamp','TCP_dst_port','UDP_dst_port','Protocol','scan_source_128','Session_ID_128','Session_ID_64','Telescope','dest_addr_type']).collect().to_pandas()

In [6]:
aggregated_df['Session_Unique'] = aggregated_df['Session_ID_128'].astype(str).str.cat(aggregated_df['Telescope'],sep='_')

In [7]:
aggregated_df['Proto'] = None
aggregated_df.loc[aggregated_df['Protocol'].isin(['TCP', 'ICMPv6', 'UDP']), 'Proto'] = aggregated_df['Protocol']

aggregated_df.loc[(aggregated_df['Protocol'] != 'UDP') & (aggregated_df['Protocol'] != 'TCP') & (aggregated_df['Protocol'] != 'ICMPv6') & (aggregated_df['TCP_dst_port'].notnull()) & (aggregated_df['UDP_dst_port'].isnull()), 'Proto'] = 'TCP'
aggregated_df.loc[(aggregated_df['Protocol'] != 'UDP') & (aggregated_df['Protocol'] != 'TCP') & (aggregated_df['Protocol'] != 'ICMPv6') & (aggregated_df['UDP_dst_port'].notnull()) & (aggregated_df['TCP_dst_port'].isnull()), 'Proto'] = 'UDP'
nan_count = aggregated_df['Proto'].isnull().sum()

In [8]:
aggregated_df.Proto.value_counts().to_frame()\
.join(round(aggregated_df.Proto.value_counts(normalize=True).to_frame()*100,1))\
.join(aggregated_df.groupby('Proto').agg(numsessions=('Session_ID_128','nunique')))\
.join(aggregated_df.groupby('Proto').agg(relsessions=('Session_ID_128',lambda s:round(s.nunique()/aggregated_df.Session_ID_128.nunique()*100,1))))\
.join(aggregated_df.groupby('Proto').agg(numsources=('scan_source_128','nunique')))\
.join(aggregated_df.groupby('Proto').agg(relsources=('scan_source_128',lambda s:round(s.nunique()/aggregated_df.scan_source_128.nunique()*100,1))))

,count,proportion,numsessions,relsessions,numsources,relsources
Proto,,,,,,
ICMPv6,33889898,66.2,132832,20.1,20373,56.5
UDP,11967255,23.4,36770,5.6,7113,19.7
TCP,5372494,10.5,614223,92.8,19977,55.4


# Table 03

In [9]:
aggregated_df.dest_addr_type.value_counts().to_frame()\
.join(round(aggregated_df.dest_addr_type.value_counts(normalize=True).to_frame()*100,2))\
.join(aggregated_df.groupby('dest_addr_type').agg(numsources=('scan_source_128','nunique')))\
.join(aggregated_df.groupby('dest_addr_type').agg(relsources=('scan_source_128',lambda s:round(s.nunique()/aggregated_df.scan_source_128.nunique()*100,2)))).rename(index={"full_zero_addr": "subnet-anycast"})

,count,proportion,numsources,relsources
dest_addr_type,,,,
randomized,32911060,64.24,2101,5.83
low-byte,11828733,23.09,32350,89.71
pattern-bytes,3054847,5.96,570,1.58
embedded-ipv4,2026762,3.96,547,1.52
subnet-anycast,1173137,2.29,1476,4.09
embedded-port,138656,0.27,80,0.22
ieee-derived,96627,0.19,26,0.07
isatap,217,0.00,2,0.01


# Table 04

## TCP Part

In [10]:
tmp = full_df.with_columns(
    udp_port = pl.when((pl.col('UDP_dst_port')>=33434) & (pl.col('UDP_dst_port')<=33523))
    .then(pl.lit('Traceroute'))
    .otherwise(pl.col('UDP_dst_port'))
)

In [11]:
totalsessions_tcp_64 = tmp.filter(~pl.col('TCP_dst_port').is_null()).select('Session_ID_64').collect().n_unique()
tmp.collect().filter(~pl.col('TCP_dst_port').is_null()).group_by('TCP_dst_port').agg(
    pl.col('Session_ID_64').n_unique().alias('Sessions /64 [#]'),
    (pl.col('Session_ID_64').n_unique()/totalsessions_tcp_64).mul(100).round(1).alias('Sessions /64 [%]'),
).sort('Sessions /64 [#]',descending=True).head(5)

TCP_dst_port,Sessions /64 [#],Sessions /64 [%]
u16,u32,f64
80,48070,87.2
443,16223,29.4
21,2592,4.7
8080,2172,3.9
22,1849,3.4


## UDP Part

In [12]:
totalsessions_udp_64 = tmp.filter(~pl.col('udp_port').is_null()).select('Session_ID_64').collect().n_unique()
tmp.collect().filter(~pl.col('udp_port').is_null()).group_by('udp_port').agg(
    pl.col('Session_ID_64').n_unique().alias('Sessions /64 [#]'),
    (pl.col('Session_ID_64').n_unique()/totalsessions_udp_64).mul(100).round(1).alias('Sessions /64 [%]'),
).sort('Sessions /64 [#]',descending=True).head(5)

udp_port,Sessions /64 [#],Sessions /64 [%]
str,u32,f64
"""Traceroute""",7067,71.4
"""53""",1945,19.7
"""161""",1718,17.4
"""500""",1710,17.3
"""123""",1669,16.9


# Table 5(a)

In [13]:
before_split = full_df.filter((pl.col('Timestamp')<datetime.datetime(2023,11,22)))

In [14]:
before_split = before_split.with_columns(
    temporal_behavior=pl.when(pl.col('is_oneoff_128'))
                              .then(pl.lit('oneoff'))
                              .when(pl.col('period_128')>0)
                              .then(pl.lit('periodic'))
                              .otherwise(pl.lit('intermittent'))
)

In [15]:
before_split.group_by('Telescope').agg(
    pl.col('scan_source_128').n_unique().alias('/128 Source addr.'),
    pl.col('scan_source_64').n_unique().alias('/64 Source addr.'),
    pl.col('AS-Number').n_unique().alias('ASN'),
    pl.col('fullhex_destination_address').n_unique().alias('Destination addr.'),
    pl.len().alias('Packets')
).sort('Telescope').collect()

Telescope,/128 Source addr.,/64 Source addr.,ASN,Destination addr.,Packets
str,u32,u32,u32,u32,u32
"""T1""",1387,1200,418,796444,2161355
"""T2""",6612,2114,479,714169,2464808
"""T3""",7,6,6,20,43
"""T4""",253,251,9,1817,3416


# Table 5(b)

## T1

In [16]:
tmp = dft1.filter((pl.col('Timestamp')<datetime.datetime(2023,11,22))).select(['Timestamp','TCP_dst_port','UDP_dst_port','Protocol','scan_source_128']).collect().to_pandas()

In [17]:
tmp['Proto'] = None
tmp.loc[tmp['Protocol'].isin(['TCP', 'ICMPv6', 'UDP']), 'Proto'] = tmp['Protocol']

tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['TCP_dst_port'].notnull()) & (tmp['UDP_dst_port'].isnull()), 'Proto'] = 'TCP'
tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['UDP_dst_port'].notnull()) & (tmp['TCP_dst_port'].isnull()), 'Proto'] = 'UDP'
nan_count = tmp['Proto'].isnull().sum()

In [18]:
tmp.groupby('Proto').agg(numsources=('scan_source_128','nunique'))\
.join(tmp.groupby('Proto').agg(relsources=('scan_source_128',lambda s:round(s.nunique()/tmp.scan_source_128.nunique()*100,1))))

,numsources,relsources
Proto,,
ICMPv6,1114,80.3
TCP,37,2.7
UDP,265,19.1


## T2

In [19]:
tmp = dft2.filter((pl.col('Timestamp')<datetime.datetime(2023,11,22))).select(['Timestamp','TCP_dst_port','UDP_dst_port','Protocol','scan_source_128']).collect().to_pandas()

In [20]:
tmp['Proto'] = None
tmp.loc[tmp['Protocol'].isin(['TCP', 'ICMPv6', 'UDP']), 'Proto'] = tmp['Protocol']

tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['TCP_dst_port'].notnull()) & (tmp['UDP_dst_port'].isnull()), 'Proto'] = 'TCP'
tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['UDP_dst_port'].notnull()) & (tmp['TCP_dst_port'].isnull()), 'Proto'] = 'UDP'
nan_count = tmp['Proto'].isnull().sum()

In [21]:
tmp.groupby('Proto').agg(numsources=('scan_source_128','nunique'))\
.join(tmp.groupby('Proto').agg(relsources=('scan_source_128',lambda s:round(s.nunique()/tmp.scan_source_128.nunique()*100,1))))

,numsources,relsources
Proto,,
ICMPv6,4112,62.2
TCP,5311,80.3
UDP,1768,26.7


## T3

In [22]:
tmp = dft3.filter((pl.col('Timestamp')<datetime.datetime(2023,11,22))).select(['Timestamp','TCP_dst_port','UDP_dst_port','Protocol','scan_source_128']).collect().to_pandas()

In [23]:
tmp['Proto'] = None
tmp.loc[tmp['Protocol'].isin(['TCP', 'ICMPv6', 'UDP']), 'Proto'] = tmp['Protocol']

tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['TCP_dst_port'].notnull()) & (tmp['UDP_dst_port'].isnull()), 'Proto'] = 'TCP'
tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['UDP_dst_port'].notnull()) & (tmp['TCP_dst_port'].isnull()), 'Proto'] = 'UDP'
nan_count = tmp['Proto'].isnull().sum()

In [24]:
tmp.groupby('Proto').agg(numsources=('scan_source_128','nunique'))\
.join(tmp.groupby('Proto').agg(relsources=('scan_source_128',lambda s:round(s.nunique()/tmp.scan_source_128.nunique()*100,1))))

,numsources,relsources
Proto,,
ICMPv6,7,100.0


## T4

In [25]:
tmp = dft4.filter((pl.col('Timestamp')<datetime.datetime(2023,11,22))).select(['Timestamp','TCP_dst_port','UDP_dst_port','Protocol','scan_source_128']).collect().to_pandas()

In [26]:
tmp['Proto'] = None
tmp.loc[tmp['Protocol'].isin(['TCP', 'ICMPv6', 'UDP']), 'Proto'] = tmp['Protocol']

tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['TCP_dst_port'].notnull()) & (tmp['UDP_dst_port'].isnull()), 'Proto'] = 'TCP'
tmp.loc[(tmp['Protocol'] != 'UDP') & (tmp['Protocol'] != 'TCP') & (tmp['Protocol'] != 'ICMPv6') & (tmp['UDP_dst_port'].notnull()) & (tmp['TCP_dst_port'].isnull()), 'Proto'] = 'UDP'
nan_count = tmp['Proto'].isnull().sum()

In [27]:
tmp.groupby('Proto').agg(numsources=('scan_source_128','nunique'))\
.join(tmp.groupby('Proto').agg(relsources=('scan_source_128',lambda s:round(s.nunique()/tmp.scan_source_128.nunique()*100,1))))

,numsources,relsources
Proto,,
ICMPv6,246,97.2
TCP,6,2.4
UDP,1,0.4


# Table 6

In [28]:
tmp = dft1.select(['Timestamp','scan_source_128','Session_ID_128','is_oneoff_128','period_128','scan_behavior_128']).collect()

In [29]:
total_sources = tmp.select('scan_source_128').n_unique()
total_sessions = tmp.select('Session_ID_128').n_unique()

In [30]:
tmp = tmp.with_columns(
    pl.when(pl.col('is_oneoff_128'))
            .then(pl.lit('One-off'))
            .when(pl.col('period_128')>0)
            .then(pl.lit('Periodic'))
            .otherwise(pl.lit('Intermittent')).alias('Temporal behavior')
)

In [31]:
tmp.group_by('Temporal behavior').agg(
    pl.col('scan_source_128').n_unique().alias('Sources [#]'),
    pl.col('scan_source_128').n_unique().truediv(total_sources).mul(100).round(2).alias('Sources [%]'),
    pl.col('Session_ID_128').n_unique().alias('Sessions [#]'),
    pl.col('Session_ID_128').n_unique().truediv(total_sessions).mul(100).round(2).alias('Sessions [%]')
).sort(by='Sources [#]',descending=True)

Temporal behavior,Sources [#],Sources [%],Sessions [#],Sessions [%]
str,u32,f64,u32,f64
"""One-off""",8244,69.71,8244,8.95
"""Intermittent""",1832,15.49,16842,18.28
"""Periodic""",1750,14.8,67067,72.78


In [32]:
tmp.group_by('scan_behavior_128').agg(
    pl.col('scan_source_128').n_unique().alias('Sources [#]'),
    pl.col('scan_source_128').n_unique().truediv(total_sources).mul(100).round(2).alias('Sources [%]'),
    pl.col('Session_ID_128').n_unique().alias('Sessions [#]'),
    pl.col('Session_ID_128').n_unique().truediv(total_sessions).mul(100).round(2).alias('Sessions [%]')
).sort(by='Sources [#]',descending=True).rename({'scan_behavior_128':'Network selection'})

Network selection,Sources [#],Sources [%],Sessions [#],Sessions [%]
cat,u32,f64,u32,f64
"""single_prefix""",10703,90.5,17939,19.47
"""independent""",1035,8.75,28433,30.85
"""inconsistent""",64,0.54,44294,48.07
"""dependent""",24,0.2,1487,1.61


# Table 7

In [33]:
rdns = pl.read_csv(RDNS_DATA_FILE,separator='|',has_header=False,new_columns=['Source_Address','ptr_record'])

In [34]:
rdns = rdns.filter(
    ~(pl.col('ptr_record').is_null()) & 
    ~(pl.col('ptr_record').str.contains(';; Warning: ID mismatch:',literal=True)) & 
    ~(pl.col('ptr_record').is_in(['No PTR record found',';; communications error to 8.8.8.8#53: timed out']))
)

In [35]:
rdns = rdns.to_pandas()

In [36]:
def get_domain(ptr,level,join=False):
    split = ptr.split('.')
    if len(split)>=level+1:
        if join:
            return '.'.join(split[-(level+1):])
        else:
            return split[-(level+1)]
    else:
        if join:
            return ptr
        else:
            return split[0]

In [37]:
rdns['sld_joined'] = rdns.ptr_record.apply(get_domain,args=(2,True))
rdns['thirdld_joined'] = rdns.ptr_record.apply(get_domain,args=(3,True))

In [38]:
rdns = pl.from_pandas(rdns)

In [39]:
rdns = rdns.with_columns(
    pl.col('Source_Address').cast(pl.Categorical)
)

In [40]:
t1 = dft1.select(['Source_Address','AS-Number','scantool','scan_source_128','Session_ID_128']).collect().join(rdns,how='left',on='Source_Address')

In [41]:
t1 = t1.with_columns(
    pl.when(pl.col('sld_joined').str.contains('caida'))
    .then(pl.lit('Caida Ark'))
    .otherwise(pl.col('scantool')).alias('final_source')
)

In [42]:
total_sources = t1.select('scan_source_128').n_unique()
total_sessions = t1.select('Session_ID_128').n_unique()

In [43]:
t1 = t1.with_columns(
    pl.col("final_source").cast(str).replace(["Trace6", "HRoute6"], ["Htrace6", "Htrace6"])
)

In [44]:
t1.group_by('final_source').agg(
    pl.col('scan_source_128').n_unique().alias('Sources [#]'),
    pl.col('scan_source_128').n_unique().truediv(total_sources).mul(100).round(2).alias('Sources [%]'),
    pl.col('Session_ID_128').n_unique().alias('Sessions [#]'),
    pl.col('Session_ID_128').n_unique().truediv(total_sessions).mul(100).round(2).alias('Sessions [%]')
).sort('Sources [#]',descending=True).filter(pl.col('final_source').is_in(['RIPEAtlasProbe','Yarrp6','Traceroute','Htrace6','6Seek','6Scan','Caida Ark'])).to_pandas().set_index('final_source').rename(index={"6Seek": "6Seeks",'Caida Ark':'CAIDA Ark'})

,Sources [#],Sources [%],Sessions [#],Sessions [%]
final_source,,,,
RIPEAtlasProbe,6484,54.83,11864,12.87
Yarrp6,22,0.19,562,0.61
Traceroute,19,0.16,163,0.18
Htrace6,9,0.08,16,0.02
6Seeks,5,0.04,17,0.02
6Scan,3,0.03,17,0.02
CAIDA Ark,2,0.02,2019,2.19


# Table 8

In [45]:
total_packets=len(t1)

In [46]:
heavy_hitters = t1.select(pl.col('scan_source_128').value_counts(sort=True,normalize=True))\
                    .unnest('scan_source_128')\
                    .filter(pl.col('proportion')>=0.1)\
                    .select(pl.col('scan_source_128')).to_series().to_list()

In [47]:
asn2type = pl.read_parquet(ASN_TYPE_FILE)

In [48]:
asn2type = asn2type.with_columns(
    (pl.col('asn').str.replace('AS','').str.to_integer().cast(pl.Int32)).alias('AS-Number')
    ).unique(subset=['AS-Number','type']).rename({'type':'asn_type'})[['AS-Number','asn_type']]

In [49]:
t1 = t1.join(asn2type, how='left', on='AS-Number')

In [50]:
tmp1 = t1.filter(~(pl.col('scan_source_128').is_in(heavy_hitters)) & (pl.col('asn_type')=='hosting')).select(
        pl.col('scan_source_128').n_unique().alias('Sources [#]'),
        pl.col('scan_source_128').n_unique().truediv(total_sources).mul(100).round(2).alias('Sources [%]'),
        pl.col('Session_ID_128').n_unique().alias('Sessions [#]'),
        pl.col('Session_ID_128').n_unique().truediv(total_sessions).mul(100).round(2).alias('Sessions [%]'),
        pl.col('Session_ID_128').len().alias('Packets [#]'),
        pl.col('Session_ID_128').len().truediv(total_packets).mul(100).round(2).alias('Packets [%]')
    ).with_columns(pl.lit("w/o Hit.").alias("asn_type")).select(["asn_type", "Sources [#]", "Sources [%]", "Sessions [#]", "Sessions [%]", "Packets [#]", "Packets [%]"])

In [51]:
tmp2 = t1.filter(~(pl.col('scan_source_128').is_in(heavy_hitters)) & (pl.col('asn_type')=='education')).select(
        pl.col('scan_source_128').n_unique().alias('Sources [#]'),
        pl.col('scan_source_128').n_unique().truediv(total_sources).mul(100).round(2).alias('Sources [%]'),
        pl.col('Session_ID_128').n_unique().alias('Sessions [#]'),
        pl.col('Session_ID_128').n_unique().truediv(total_sessions).mul(100).round(2).alias('Sessions [%]'),
        pl.col('Session_ID_128').len().alias('Packets [#]'),
        pl.col('Session_ID_128').len().truediv(total_packets).mul(100).round(2).alias('Packets [%]')
    ).with_columns(pl.lit("w/o Hit.").alias("asn_type")).select(["asn_type", "Sources [#]", "Sources [%]", "Sessions [#]", "Sessions [%]", "Packets [#]", "Packets [%]"])

In [52]:
tmp3 = t1.group_by('asn_type').agg(
    pl.col('scan_source_128').n_unique().alias('Sources [#]'),
    pl.col('scan_source_128').n_unique().truediv(total_sources).mul(100).round(2).alias('Sources [%]'),
    pl.col('Session_ID_128').n_unique().alias('Sessions [#]'),
    pl.col('Session_ID_128').n_unique().truediv(total_sessions).mul(100).round(2).alias('Sessions [%]'),
    pl.col('Session_ID_128').len().alias('Packets [#]'),
    pl.col('Session_ID_128').len().truediv(total_packets).mul(100).round(2).alias('Packets [%]')
).sort('Sources [#]',descending=True)

In [53]:
table_df = tmp3.vstack(tmp1).vstack(tmp2)\
        .sort('Sources [#]', descending=True)\
        .to_pandas().set_index('asn_type').rename(index={None:'unknown'})

if 'unknown' in table_df.index:
    unknown_row = table_df.loc[['unknown']]
    table_df = table_df.drop(index='unknown')
    table_df = pd.concat([table_df, unknown_row])

In [54]:
table_df

,Sources [#],Sources [%],Sessions [#],Sessions [%],Packets [#],Packets [%]
asn_type,,,,,,
hosting,6624,56.01,23682,25.70,28371475,65.06
w/o Hit.,6621,55.99,23674,25.69,4496454,10.31
isp,4681,39.58,46864,50.85,1478591,3.39
education,245,2.07,17634,19.14,13629270,31.25
w/o Hit.,244,2.06,17627,19.13,4375030,10.03
business,194,1.64,2259,2.45,71689,0.16
government,6,0.05,7,0.01,96,0.00
unknown,76,0.64,1707,1.85,58527,0.13
